In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 178ms/step - loss: 3.1556 - accuracy: 0.5148
Epoch 2/30
58/58 [==============================] - 10s 178ms/step - loss: 2.1184 - accuracy: 0.5696
Epoch 3/30
58/58 [==============================] - 10s 176ms/step - loss: 1.6900 - accuracy: 0.5805
Epoch 4/30
58/58 [==============================] - 10s 179ms/step - loss: 1.4521 - accuracy: 0.6648
Epoch 5/30
58/58 [==============================] - 10s 179ms/step - loss: 1.3227 - accuracy: 0.6791
Epoch 6/30
58/58 [==============================] - 10s 178ms/step - loss: 1.2045 - accuracy: 0.7010
Epoch 7/30
58/58 [==============================] - 10s 178ms/step - loss: 1.1004 - accuracy: 0.7448
Epoch 8/30
58/58 [==============================] - 10s 178ms/step - loss: 1.0203 - accuracy: 0.7579
Epoch 9/30
58/58 [==============================] - 10s 179ms/step - loss: 0.9657 - accuracy: 0.7700
Epoch 10/30
58/58 [==============================] - 10s 179ms/step - loss: 0.9023 - accura

58/58 [==============================] - 11s 184ms/step - loss: 0.6370 - accuracy: 0.8193
Epoch 22/30
58/58 [==============================] - 11s 182ms/step - loss: 0.6276 - accuracy: 0.8346
Epoch 23/30
58/58 [==============================] - 10s 180ms/step - loss: 0.6188 - accuracy: 0.8368
Epoch 24/30
58/58 [==============================] - 10s 178ms/step - loss: 0.9739 - accuracy: 0.5827
Epoch 25/30
58/58 [==============================] - 10s 180ms/step - loss: 0.9646 - accuracy: 0.4962
Epoch 26/30
58/58 [==============================] - 11s 182ms/step - loss: 0.9284 - accuracy: 0.5268
Epoch 27/30
58/58 [==============================] - 10s 180ms/step - loss: 0.9072 - accuracy: 0.5378
Epoch 28/30
58/58 [==============================] - 11s 185ms/step - loss: 0.8898 - accuracy: 0.5367
Epoch 29/30
58/58 [==============================] - 11s 182ms/step - loss: 0.8763 - accuracy: 0.5323
Epoch 30/30
58/58 [==============================] - 11s 182ms/step - loss: 0.8658 - accuracy:

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 19s 323ms/step - loss: 4.4222 - accuracy: 0.4940
Epoch 2/30
58/58 [==============================] - 19s 321ms/step - loss: 2.6450 - accuracy: 0.5465
Epoch 3/30
58/58 [==============================] - 19s 329ms/step - loss: 2.0241 - accuracy: 0.5422
Epoch 4/30
58/58 [==============================] - 19s 327ms/step - loss: 1.7456 - accuracy: 0.5685
Epoch 5/30
58/58 [==============================] - 19s 327ms/step - loss: 1.5909 - accuracy: 0.6375
Epoch 6/30
58/58 [==============================] - 19s 322ms/step - loss: 1.4498 - accuracy: 0.6846
Epoch 7/30
58/58 [==============================] - 19s 322ms/step - loss: 1.3290 - accuracy: 0.6857
Epoch 8/30
58/58 [==============================] - 19s 321ms/step - loss: 1.2417 - accuracy: 0.6966
Epoch 9/30
58/58 [==============================] - 19s 324ms/step - loss: 1.1808 - accuracy: 0.7207
Epoch 10/30
58/58 [==============================] - 19s 326ms/step - loss: 1.1168 - accura

58/58 [==============================] - 18s 304ms/step - loss: 0.8120 - accuracy: 0.6287
Epoch 22/30
58/58 [==============================] - 18s 315ms/step - loss: 0.7228 - accuracy: 0.7525
Epoch 23/30
58/58 [==============================] - 18s 315ms/step - loss: 0.7704 - accuracy: 0.6758
Epoch 24/30
58/58 [==============================] - 18s 318ms/step - loss: 0.7154 - accuracy: 0.7229
Epoch 25/30
58/58 [==============================] - 18s 317ms/step - loss: 0.7011 - accuracy: 0.7547
Epoch 26/30
58/58 [==============================] - 18s 315ms/step - loss: 0.6788 - accuracy: 0.7547
Epoch 27/30
58/58 [==============================] - 18s 315ms/step - loss: 0.6570 - accuracy: 0.7645
Epoch 28/30
58/58 [==============================] - 18s 316ms/step - loss: 0.6453 - accuracy: 0.7568
Epoch 29/30
58/58 [==============================] - 18s 314ms/step - loss: 0.7373 - accuracy: 0.7010
Epoch 30/30
58/58 [==============================] - 18s 316ms/step - loss: 0.6904 - accuracy: